Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic Regression with Regularization

In [5]:
batch_size = 128

graph_logreg = tf.Graph()
with graph_logreg.as_default():

  # Input data. 
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
#   norm_weights = 0.01 * tf.nn.l2_loss(weights)
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
#   loss_2 = tf.reduce_mean(norm_weights)
  loss_2 = 0.1 * tf.nn.l2_loss(weights) / batch_size
  loss = loss + loss_2
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

## Neural Network with Regularization 

In [8]:
batch_size = 128
hidden_nodes = 1024

graph_nn = tf.Graph()

with graph_nn.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights1 = tf.Variable(
                tf.truncated_normal([image_size * image_size, hidden_nodes]))
    weights2 = tf.Variable(
                tf.truncated_normal([hidden_nodes, num_labels]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
#     weights_concat = tf.concat(0,[weights1,tf.transpose(weights2)])
#     norm_weights = 0.01 * tf.nn.l2_loss(weights_concat)
    
    # Training
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
    logits = tf.matmul(hidden,weights2)+biases2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                            logits, tf_train_labels))
#     loss_2 = tf.reduce_mean(norm_weights)
    loss_2 = 0.1 * (tf.nn.l2_loss(weights1) + 
                   tf.nn.l2_loss(weights2)) / batch_size
    loss = loss + loss_2
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
                        tf.matmul(
                            tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),
                            weights2)
                        +biases2)
    test_prediction = tf.nn.softmax(
                        tf.matmul(
                            tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),
                            weights2)
                        +biases2)
    

In [9]:
#Let's' run it
graph_selected = graph_nn

In [10]:
num_steps = 3001

with tf.Session(graph=graph_selected) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 606.485229
Minibatch accuracy: 10.2%
Validation accuracy: 31.0%
Minibatch loss at step 500: 180.848236
Minibatch accuracy: 80.5%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 112.891235
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 74.289612
Minibatch accuracy: 88.3%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 49.786633
Minibatch accuracy: 90.6%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 33.794441
Minibatch accuracy: 87.5%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 23.016510
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Test accuracy: 92.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [12]:
batch_size = 128
hidden_nodes = 1024

graph_nn = tf.Graph()

with graph_nn.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights1 = tf.Variable(
                tf.truncated_normal([image_size * image_size, hidden_nodes]))
    weights2 = tf.Variable(
                tf.truncated_normal([hidden_nodes, num_labels]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
#     weights_concat = tf.concat(0,[weights1,tf.transpose(weights2)])
#     norm_weights = 0.01 * tf.nn.l2_loss(weights_concat)
    
    # Training
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
    logits = tf.matmul(hidden,weights2)+biases2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                            logits, tf_train_labels))
#     loss_2 = tf.reduce_mean(norm_weights)
    loss_2 = 0.1 * (tf.nn.l2_loss(weights1) + 
                   tf.nn.l2_loss(weights2)) / batch_size
    loss = loss + loss_2
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
                        tf.matmul(
                            tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),
                            weights2)
                        +biases2)
    test_prediction = tf.nn.softmax(
                        tf.matmul(
                            tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),
                            weights2)
                        +biases2)
    
    num_steps = 3001

with tf.Session(graph=graph_nn) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
#     offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = batch_size * np.random.choice(np.arange(5))
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 620.817566
Minibatch accuracy: 9.4%
Validation accuracy: 21.6%
Minibatch loss at step 500: 165.746994
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 112.143127
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 1500: 75.871681
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 2000: 51.337276
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 2500: 34.730930
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 3000: 23.500076
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Test accuracy: 84.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [46]:
batch_size = 128
hidden_nodes = 1024

graph_nn_dropout = tf.Graph()

with graph_nn_dropout.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights1 = tf.Variable(
                tf.truncated_normal([image_size * image_size, hidden_nodes],stddev=0.03))
    weights2 = tf.Variable(
                tf.truncated_normal([hidden_nodes, num_labels],stddev=0.03))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    keep_prob = 0.5
    drop1 = tf.nn.dropout(tf_train_dataset,keep_prob)
#     drop1 = tf_train_dataset
    activations = tf.nn.relu(tf.matmul(drop1,weights1)+biases1)
    drop2 = tf.nn.dropout(activations,keep_prob)
    
    # Training
    logits = tf.matmul(drop2,weights2)+biases2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                            logits, tf_train_labels))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
                        tf.matmul(
                            tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),
                            weights2)
                        +biases2)
    test_prediction = tf.nn.softmax(
                        tf.matmul(
                            tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),
                            weights2)
                        +biases2)

In [47]:
num_steps = 3001

with tf.Session(graph=graph_nn_dropout) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
#     offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = batch_size * np.random.choice(np.arange(5))
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.409577
Minibatch accuracy: 12.5%
Validation accuracy: 33.6%
Minibatch loss at step 500: 0.043598
Minibatch accuracy: 99.2%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 0.026279
Minibatch accuracy: 99.2%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 0.004646
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 0.004877
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 2500: 0.001299
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 3000: 0.003710
Minibatch accuracy: 100.0%
Validation accuracy: 80.4%
Test accuracy: 87.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [58]:
batch_size = 128
hidden_nodes = 1024

graph_nn_3 = tf.Graph()

with graph_nn_3.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights1 = tf.Variable(
                tf.truncated_normal([image_size * image_size, hidden_nodes],stddev=0.01))
    weights2 = tf.Variable(
                tf.truncated_normal([hidden_nodes, hidden_nodes],stddev=0.01))
    weights3 = tf.Variable(
                tf.truncated_normal([hidden_nodes, num_labels],stddev=0.01))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    biases2 = tf.Variable(tf.zeros([hidden_nodes]))
    biases3 = tf.Variable(tf.zeros([num_labels]))
    
    keep_prob = 0.5
    drop0 = tf.nn.dropout(tf_train_dataset,keep_prob)
#     drop1 = tf_train_dataset
    activations1 = tf.nn.relu(tf.matmul(drop0,weights1)+biases1)
    drop1 = tf.nn.dropout(activations1,keep_prob)
    activations2 = tf.nn.relu(tf.matmul(drop1,weights2)+biases2)
    drop2 = tf.nn.dropout(activations2,keep_prob)
#     activations3 = tf.nn.relu(tf.matmul(drop2,weights3)+biases3)
    
    # Training
    logits = tf.matmul(drop2,weights3)+biases3
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                            logits, tf_train_labels))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset,weights1)+biases1)
    valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1,weights2)+biases2)
    valid_prediction = tf.nn.softmax(
                        tf.matmul(valid_hidden2,weights3)+biases3)
    
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset,weights1)+biases1)
    test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1,weights2)+biases2)
    test_prediction = tf.nn.softmax(
                        tf.matmul(test_hidden2,weights3)+biases3)

In [105]:
batch_size = 128
hidden_nodes_1 = 1024
hidden_nodes_2 = 500
hidden_nodes_3 = 50

graph_nn_4 = tf.Graph()

with graph_nn_4.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights1 = tf.Variable(
                tf.truncated_normal([image_size * image_size, hidden_nodes_1],stddev=0.01))
    weights2 = tf.Variable(
                tf.truncated_normal([hidden_nodes_1, hidden_nodes_2],stddev=0.01))
    weights3 = tf.Variable(
                tf.truncated_normal([hidden_nodes_2, hidden_nodes_3],stddev=0.01))
    weights4 = tf.Variable(
                tf.truncated_normal([hidden_nodes_3, num_labels],stddev=0.01))
    biases1 = tf.Variable(tf.zeros([hidden_nodes_1]))
    biases2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    biases3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    biases4 = tf.Variable(tf.zeros([num_labels]))
    
    keep_prob = 0.5
    drop0 = tf.nn.dropout(tf_train_dataset,keep_prob)
#     drop1 = tf_train_dataset
    activations1 = tf.nn.relu(tf.matmul(drop0,weights1)+biases1)
    drop1 = tf.nn.dropout(activations1,keep_prob)
    activations2 = tf.nn.relu(tf.matmul(drop1,weights2)+biases2)
    drop2 = tf.nn.dropout(activations2,keep_prob)
    activations3 = tf.nn.relu(tf.matmul(drop2,weights3)+biases3)
    drop3 = tf.nn.dropout(activations3,keep_prob)
    
    # Training
    logits = tf.matmul(drop3,weights4)+biases4
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                            logits, tf_train_labels))
    
    # Optimizer
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.2, global_step, 1000, 0.86, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
#     optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset,weights1)+biases1)
    valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1,weights2)+biases2)
    valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2,weights3)+biases3)
    valid_prediction = tf.nn.softmax(
                        tf.matmul(valid_hidden3,weights4)+biases4)
    
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset,weights1)+biases1)
    test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1,weights2)+biases2)
    test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2,weights3)+biases3)
    test_prediction = tf.nn.softmax(
                        tf.matmul(test_hidden3,weights4)+biases4)

In [106]:
num_steps = 24001

with tf.Session(graph=graph_nn_4) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
#     offset = batch_size * np.random.choice(np.arange(5))
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.302577
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 500: 1.884682
Minibatch accuracy: 18.0%
Validation accuracy: 27.3%
Minibatch loss at step 1000: 0.806530
Minibatch accuracy: 75.8%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 0.507107
Minibatch accuracy: 82.8%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.519289
Minibatch accuracy: 87.5%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 0.570178
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 0.626888
Minibatch accuracy: 81.2%
Validation accuracy: 85.5%
Minibatch loss at step 3500: 0.528848
Minibatch accuracy: 83.6%
Validation accuracy: 85.9%
Minibatch loss at step 4000: 0.538467
Minibatch accuracy: 82.8%
Validation accuracy: 86.1%
Minibatch loss at step 4500: 0.557504
Minibatch accuracy: 82.8%
Validation accuracy: 86.2%
Minibatch loss at step 5000: 0.504562
Minibatch accuracy: 85.2%
Validation accurac